# PassManager Results

Recently, IBM Quantum introduced the **PassManager** and the AI Routing Pass as part of the transpiler passes designed to optimize quantum circuit execution on IBM QPUs. Qiskit-P allows implemention of the SWAP network strategy directly in the transpilation sequence, and Qiskit-AI uses machine-learning techniques to predict the best qubit mapping and swap strategy, reducing circuit depth and two-qubit gate count. Qiskit (v1.4.2) is employed for quantum circuit construction and transpilation.

In [1]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
import networkx as nx
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector, Parameter
import time

In [2]:
def build_max_cut_paulis(graph: nx.Graph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in graph.edges():
        paulis = ["I"] * len(graph)
        paulis[edge[0]], paulis[edge[1]] = "Z", "Z"

        weight = graph.get_edge_data(edge[0], edge[1]).get("weight", 1.0)

        pauli_list.append(("".join(paulis)[::-1], weight))

    return pauli_list

In [3]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    HighLevelSynthesis,
    InverseCancellation
)

from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import (
    SwapStrategy,
    FindCommutingPauliEvolutions,
    Commuting2qGateRouter,
)
from qiskit.circuit.library.standard_gates.equivalence_library import _sel
from qiskit.circuit.library import CXGate
from qiskit import transpile


In [5]:
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.converters import circuit_to_dag, dag_to_circuit

class QAOAPass(TransformationPass):

    def __init__(self, num_layers, num_qubits, init_state = None, mixer_layer = None):

        super().__init__()
        self.num_layers = num_layers
        self.num_qubits = num_qubits
        
        if init_state is None:
            # Add default initial state -> equal superposition
            self.init_state = QuantumCircuit(num_qubits)
            self.init_state.h(range(num_qubits))
        else: 
            self.init_state = init_state
        
        if mixer_layer is None:
            # Define default mixer layer
            self.mixer_layer = QuantumCircuit(num_qubits)
            beta = Parameter("beta")
            self.mixer_layer.rx(-2*beta, range(num_qubits))
        else:
            self.mixer_layer = mixer_layer

    def run(self, cost_layer_dag):

        cost_layer = dag_to_circuit(cost_layer_dag)
        qaoa_circuit = QuantumCircuit(self.num_qubits, self.num_qubits)
        # Re-parametrize the circuit
        gammas = ParameterVector("γ", self.num_layers)
        betas = ParameterVector("β", self.num_layers)

        # Add initial state
        qaoa_circuit.compose(self.init_state, inplace = True)

        # iterate over number of qaoa layers
        # and alternate cost/reversed cost and mixer
        for layer in range(self.num_layers): 
        
            bind_dict = {cost_layer.parameters[0]: gammas[layer]}
            bound_cost_layer = cost_layer.assign_parameters(bind_dict)
            
            bind_dict = {self.mixer_layer.parameters[0]: betas[layer]}
            bound_mixer_layer = self.mixer_layer.assign_parameters(bind_dict)
        
            if layer % 2 == 0:
                # even layer -> append cost
                qaoa_circuit.compose(bound_cost_layer, range(self.num_qubits), inplace=True)
            else:
                # odd layer -> append reversed cost
                qaoa_circuit.compose(bound_cost_layer.reverse_ops(), range(self.num_qubits), inplace=True)
        
            # the mixer layer is not reversed
            qaoa_circuit.compose(bound_mixer_layer, range(self.num_qubits), inplace=True)
        

        if self.num_layers % 2 == 1:
            # iterate over layout permutations to recover measurements
            if self.property_set["virtual_permutation_layout"]:
                for cidx, qidx in self.property_set["virtual_permutation_layout"].get_physical_bits().items():
                    qaoa_circuit.measure(qidx, cidx)
            else:
                # print("layout not found, assigining trivial layout")
                for idx in range(self.num_qubits):
                    qaoa_circuit.measure(idx, idx)
        else:
            for idx in range(self.num_qubits):
                qaoa_circuit.measure(idx, idx)
    
        return circuit_to_dag(qaoa_circuit)
        

In [6]:
seed = 1
nq = 0
nqs = [20,30,40,50,60,70,80,90,100,110,120]
nqs = [20]
seeds = [1,2,3]
method_name = "PassManager"
for seed in seeds:
    graphs = np.load(f"./Data/problems_transpilers_{seed}.npy", allow_pickle=True).item()
    for nq in nqs:
        operations = np.load(f"./Data/{nq}_transpiler_{seed}.npy", allow_pickle=True).item()
        probs = graphs["probs"][nq]
        for Ed in probs:
            G = graphs["G"][nq][Ed]
            ti = time.time()
            local_correlators = build_max_cut_paulis(G)
            cost_operator = SparsePauliOp.from_list(local_correlators)

            dummy_initial_state = QuantumCircuit(nq)  # the real initial state is defined later
            dummy_mixer_operator = QuantumCircuit(nq)

            cost_layer = QAOAAnsatz(
                cost_operator,
                reps=1,
                initial_state=dummy_initial_state,
                mixer_operator=dummy_mixer_operator,
                name="QAOA cost block",
            )
            swap_strategy = SwapStrategy.from_line([i for i in range(nq)])
            edge_coloring = {(idx, idx + 1): (idx + 1) % 2 for idx in range(nq)}
            pre_init = PassManager(
                        [HighLevelSynthesis(basis_gates=['PauliEvolution']),
                        FindCommutingPauliEvolutions(),
                        Commuting2qGateRouter(
                                swap_strategy,
                                edge_coloring,
                            ),
                        HighLevelSynthesis(basis_gates=["x", "cx", "sx", "rz", "id", "cz", "rx"]),
                        InverseCancellation(gates_to_cancel=[CXGate()]),
                        ]
            )
            tmp = pre_init.run(cost_layer)
            layout = [(i,i+1) for i in range(nq-1)] + [(i+1,i) for i in range(nq-1)]
            init = PassManager([QAOAPass(num_layers=1, num_qubits=nq)])
            qc = init.run(tmp)
            tf = time.time()
            print("Operations:", qc.count_ops())

            operations[method_name][Ed] = qc.count_ops()
            operations[method_name][Ed]["time"] = tf - ti
            operations[method_name][Ed]["depth"] = qc.depth()


        np.save(f"./Data/{nq}_transpiler_{seed}.npy", operations)

Operations: OrderedDict([('cx', 519), ('rz', 41), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 527), ('rz', 57), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 527), ('rz', 74), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 525), ('rz', 91), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 537), ('rz', 107), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 535), ('rz', 124), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 545), ('rz', 141), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 543), ('rz', 157), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 547), ('rz', 174), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 551), ('rz', 190), ('h', 20), ('rx', 20), ('measure', 20)])
Operations: OrderedDict([('cx', 519), ('rz', 41), ('h', 20), ('rx', 20), ('measure',